# Exploratory Data Analyst : Case Mall Customers
---
Analisis menggunakan datasheet Mall Customer untuk menemukan pola konsumen yang paling sering berbelanja berdasarkan jenis kelamin, pendapatan tahunan, skor pengeluaran, kategori usia dan lain-lain 

# 1. Load datasheet from postgresql 
---
pertama kita akan membuat koneksi database ke postgresql pada tabel customer_segmentation untuk datasheet yang akan kita gunakan dalam proses analisis 

In [16]:
import pandas as pd # library untuk pengolohan dataframe 
import numpy as np  # library untuk komputasi numerik 

# import library untuk koneksi ke database postgresql 
from sqlalchemy import create_engine

# menyambungkan ke database postgresql 
# Pastikan Anda mengganti parameter sesuai dengan kredensial PostgreSQL Anda
db_params = {
    'host': 'localhost',
    'dbname': 'DB_ML',
    'user': 'postgres',
    'password': 'admin',
    'port': 5432  # default PostgreSQL port
}

# membuat engine untuk koneksi menggunakan SQL Alchemy 
engine = create_engine(f'postgresql://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}:{db_params["port"]}/{db_params["dbname"]}')

# muat tabel ke dalam dataframe pandas 
df = pd.read_sql_table('customer_segmentation', engine)

# tampilkan isi dari tabel customer segemntation 
print(df.head())

print("""
Dari datasheet tersebut dapat diketahui bahwa kita bisa melakukan analisis 
untuk menemukan insight dengan memanfaatkan kolom annual income dan spending score 
terhadap gender dan kategori usia sehingga hasil akhirnya kita bisa membuat 
analisis customer segmentation dan meningkatkan penjualan.
""")

   CustomerID  Gender  Age  Annual Income (k$)  Spending Score (1-100)  \
0           1    Male   19                  15                      39   
1           2    Male   21                  15                      81   
2           3  Female   20                  16                       6   
3           4  Female   23                  16                      77   
4           5  Female   31                  17                      40   

  kategori_usia  
0   Dewasa Muda  
1   Dewasa Muda  
2   Dewasa Muda  
3   Dewasa Muda  
4        Dewasa  

Dari datasheet tersebut dapat diketahui bahwa kita bisa melakukan analisis 
untuk menemukan insight dengan memanfaatkan kolom annual income dan spending score 
terhadap gender dan kategori usia sehingga hasil akhirnya kita bisa membuat 
analisis customer segmentation dan meningkatkan penjualan.



## 1.1 Melihat komposisi dari datasheet yang digunakan
--- 
pertama kita akan melihat komposisi dari datasheet yang kita gunakan seperti jumlah baris dan kolom, dan tipe data serta mengecek apakah ada data yang nilai nya mengalami missing value dan lain-lain

In [20]:
print(f'Jumlah baris pada datasheet tersebut berjumlah : {df.shape[0]} baris') # mengecek jumlah baris pada datasheet
print(f'Jumlah kolom pada datasheet tersebut berjumlah : {df.shape[1]} kolom') # mengecek jumlah kolom pada datasheet 

print("""
ternyata jumlah komposisi pada datasheet kita berjumlah 200 baris data dan 6 kolom meskipun
jumlahnya sedikit kita akan memaksimalkan denganmenggunakan metode statistik yang sesuai agar 
memberikan haisl yang optimal 
""")

Jumlah baris pada datasheet tersebut berjumlah : 200 baris
Jumlah kolom pada datasheet tersebut berjumlah : 6 kolom

ternyata jumlah komposisi pada datasheet kita berjumlah 200 baris data dan 6 kolom meskipun
jumlahnya sedikit kita akan memaksimalkan denganmenggunakan metode statistik yang sesuai agar 
memberikan haisl yang optimal 



## 1.2 Mengecek apakah ada data yang bernilai missing value
---
pada bagian ini kita akan melakukan pengecekan untuk mengetahui apakah ada data yang mengalami missing value pada datasheet yang kita gunakan dan menghapus atau mengganti nilai yang missing value tersebut sebelum dilanjutkan ke dalam proses berikut nya 

In [22]:
# mengecek apakah ada data yang mengalami missing value 
print(f'jumlah duplikasi : {df.duplicated().sum()} data')

print("""
ternyata tidak ada data yang mengalami misisng value sehingga kita bisa lanjutkan ke tahap berikutnya 
yaitu analisis outlier untuk mengetahui bahwa apakah ada fitur yang mengalami outliers yang mungkin 
bisa mempengarui kinerja dari model yang kita buat nanti
""")

jumlah duplikasi : 0 data

ternyata tidak ada data yang mengalami misisng value sehingga kita bisa lanjutkan ke tahap berikutnya 
yaitu analisis outlier untuk mengetahui bahwa apakah ada fitur yang mengalami outliers yang mungkin 
bisa mempengarui kinerja dari model yang kita buat nanti



# 2.Exploratory data analyst (EDA)
---
Tahap selanjutnya kita akan masuk kedalam tahap analisis EDA untuk melakukan preprocessing pada data yang meliputi pembersihan pada data dan mencari insight yang mungkin bisa memberikan keuntungan kedepan nya 

## 2.1 Rename columns
---
pada proses ini kita akan melakukan perubahan nama fitur pada kolom datasheet kita agar mudah untuk melakukan analisis kedepannya 

In [25]:
# rename kolom annual income dan spending skor 
df = df.rename(columns={'Annual Income (k$)':'Pendapatan_Tahunan', 'Spending Score (1-100)':'Pengeluaran_Tahunan'})
print(df.head())

print("""
dengan merubah nama kolom annual income (k$) dan spending score (1-100) kita bisa lebih mudah 
untuk melakukan analisis jika kita menggunakan 2 kolom tersebut nanti nya 
""")

   CustomerID  Gender  Age  Pendapatan_Tahunan  Pengeluaran Tahunan  \
0           1    Male   19                  15                   39   
1           2    Male   21                  15                   81   
2           3  Female   20                  16                    6   
3           4  Female   23                  16                   77   
4           5  Female   31                  17                   40   

  kategori_usia  
0   Dewasa Muda  
1   Dewasa Muda  
2   Dewasa Muda  
3   Dewasa Muda  
4        Dewasa  

dengan merubah nama kolom annual income (k$) dan spending score (1-100) kita bisa lebih mudah 
untuk melakukan analisis jika kita menggunakan 2 kolom tersebut nanti nya 



## 2.2 Analisis data outlier dan visualisasi boxplot
---
selanjutnya kita akan melakukan visulisasi data dengan boxplot untuk melihat penyebaran dari dari fitur data yang kita miliki 